In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 85))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp
import threading

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
            x = requests.get(uri).text
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            uri = 'https://' + '/'.join(mod[:-1]) + '/raw/main/' + mod[-1] + '.py'
            x = requests.get(uri).text
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except Exception as err:
        print(err)

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()

def img2b64(x):
    if isinstance(x, bytes):
        return base64.b64encode(x).decode()
    elif isinstance(x, str):
        with open(x, 'rb') as fr:
            return base64.b64encode(fr.read()).decode()
    raise


###
### Display ###
###

_IMPORT_('import cv2')
_IMPORT_('from PIL import Image')
_IMPORT_('from torchvision.utils import make_grid')
_IMPORT_('import matplotlib.pyplot as plt')
_IMPORT_('import plotly')
_IMPORT_('import plotly.graph_objects as go')
_IMPORT_('import ipywidgets as widgets')
_IMPORT_('from ipywidgets import interact, interactive, fixed, interact_manual')

# plotly.offline.init_notebook_mode(connected=False)

plt.rcParams['figure.figsize'] = (12.0, 8.0)

def show_table(headers, data, width=900):
    ncols = len(headers)
    width = int(width / ncols)
    lralign = []
    caption = []
    for item in headers:
        astr = ''
        if item[0] == ':':
            astr = ':'
            item = item[1:]
        astr += '---'
        if item[-1] == ':':
            astr += ':'
            item = item[:-1]
        lralign.append(astr)
        caption.append(item)
    captionstr = '|'.join(caption) + chr(10)
    lralignstr = '|'.join(lralign) + chr(10)
    imgholdstr = '|'.join(['<img width=%d/>' % width] * ncols) + chr(10)
    table = captionstr + lralignstr + imgholdstr
    is_dict = isinstance(data[0], dict)
    for row in data:
        if is_dict:
            table += '|'.join([f'{row[c]}' for c in caption]) + chr(10)
        else:
            table += '|'.join([f'{col}' for col in row]) + chr(10)
    return Markdown(table)

def show_video(vidsrc, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if vidsrc.startswith('http'):
        data_url = vidsrc
    else:
        mp4 = open(vidsrc, 'rb').read()
        data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<video %s %s controls src="%s" type="video/mp4"/>' % (W, H, data_url))

def show_image(imgsrc, width=None, height=None):
    if isinstance(imgsrc, np.ndarray):
        img = imgsrc
        if width or height:
            if width and height:
                size = (width, height)
            else:
                rate = img.shape[1] / img.shape[0]
                if width:
                    size = (width, int(width/rate))
                else:
                    size = (int(height*rate), height)
            img = cv2.resize(img, size)
            plt.figure(figsize=(3*int(size[0]/80+1), 3*int(size[1]/80+1)), dpi=80)
        plt.axis('off')
        if len(img.shape) > 2:
            plt.imshow(img);
        else:
            plt.imshow(img, cmap='gray');
        return

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if imgsrc.startswith('http'):
        data_url = imgsrc
    else:
        if len(imgsrc) > 2048:
            data_url = 'data:image/jpg;base64,' + imgsrc
        else:
            img = open(imgsrc, 'rb').read()
            data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if image_path.startswith('http'):
        data_url = image_path
    else:
        img = open(image_path, 'rb').read()
        data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

def im_read(url, rgb=True, size=None):
    if url.startswith('http'):
        response = requests.get(url)
        if response:
            imgmat = np.frombuffer(response.content, dtype=np.uint8)
            img = cv2.imdecode(imgmat, cv2.IMREAD_COLOR)
        else:
            return None
    else:
        img = cv2.imread(url)
        
    if rgb:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if size:
        if isinstance(size, int):
            size = (size, size)
        img = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    return img

CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed


## 引入工具

In [2]:
# _IMPORT_('import gitee.com/qrsforever/nb_easy/easy_widget')
_IMPORT_('import /data/nb_easy/easy_widget')

In [3]:
import xlrd
from minio import Minio
oss_client = Minio(
    endpoint=os.environ.get('MINIO_SERVER_URL'),
    access_key=os.environ.get('MINIO_ACCESS_KEY'),
    secret_key=os.environ.get('MINIO_SECRET_KEY'),
    secure=True)

def oss_get_bypath(path):
    objs = oss_client.list_objects('frepai', path, recursive=False)
    options = []
    for o in objs:
        if 'outputs' in o.object_name:
            continue
        object_path = o.object_name[:-1]
        options.append((os.path.basename(object_path), object_path))
    if len(options) == 0:
        options = [('NONE', 'NONE')]
    return options

def oss_get_video_list(prefix):
    objs = oss_client.list_objects('frepai', f'{prefix}/videos/', recursive=False)
    options = []
    for o in objs:
        options.append((os.path.basename(o.object_name)[8:], 'https://frepai.s3.didiyunapi.com/' + o.object_name))
    if len(options) == 0:
        options = [('NONE', 'NONE')]
    return options

def oss_get_video_samples(prefix):
    if prefix[-1] != '/':
        prefix += '/'
    objs = oss_client.list_objects('frepai', prefix, recursive=False)
    options = []
    for o in objs:
        options.append((os.path.basename(o.object_name), 'https://frepai.s3.didiyunapi.com/' + o.object_name))
    if len(options) == 0:
        options = [('NONE', 'NONE')]
    return options

def oss_remove_outputs():
    objs = oss_client.list_objects('frepai', 'datasets/ladder/outputs/', recursive=True)
    for o in objs:
        oss_client.remove_object('frepai', o.object_name)
        
def parse_xls_report():
    for file in os.listdir('report'):
        if not file.endswith('xls'):
            continue
        file = f'report/{file}'
        data = xlrd.open_workbook(file)
        table = data.sheets()[0]
        for i in range(table.nrows):
            rowdata = table.row_values(i)
            if '摄像头' in rowdata:
                taskt_col = rowdata.index('任务类型')
                count_col = rowdata.index('审核次数')
                video_col = rowdata.index('视频源地址')
                taskt_info = table.col_values(taskt_col)[i+1:]
                count_info = table.col_values(count_col)[i+1:]
                video_info = table.col_values(video_col)[i+1:]
                for task, count, url in zip(taskt_info, count_info, video_info):
                    if task and count and url:
                        filename = os.path.basename(url).replace('.mp4', f'_{int(count)}.mp4')
                        result = requests.get(url.replace('s3', 's3-internal'))
                        oss_client.put_object(
                            'frepai', f'datasets/ladder/{task}/{filename}',
                            io.BytesIO(result.content), len(result.content), content_type='video/mp4')
                break
        os.remove(file)

## 设备信息

In [4]:
devices = (
    ["00e685ef87b4", "压弯", "SSAC-462825-DBFBF"],
    ["00ee44217446", "弯折", "SSSS-120078-DBAFF"],
    ["00cf60a8f512", "切割", "SSAC-332308-DCBEC"],
    ["004309167638", "磨擦", "SSAC-292214-CEDDE"],
    ["006b246f536d", "打孔", "SSAC-292195-CEDDE"],
    ["0031059d1d11", "站冲", "SSAC-292202-FBDEB"],
    ["002b359e3931", "组装", "SSAC-292197-ECFAB"],
    ["00232ee8876d", "焊接", "SSAC-292217-AAFFA"],
    ["00047dd87188", "翻边", "SSAC-292170-ECFAB"],
)

show_table(['MAC', 'Label', 'UUID'], devices)

MAC|Label|UUID
---|---|---
<img width=300 height=1/>|<img width=300 height=1/>|<img width=300 height=1/>
00e685ef87b4|压弯|SSAC-462825-DBFBF
00ee44217446|弯折|SSSS-120078-DBAFF
00cf60a8f512|切割|SSAC-332308-DCBEC
004309167638|磨擦|SSAC-292214-CEDDE
006b246f536d|打孔|SSAC-292195-CEDDE
0031059d1d11|站冲|SSAC-292202-FBDEB
002b359e3931|组装|SSAC-292197-ECFAB
00232ee8876d|焊接|SSAC-292217-AAFFA
00047dd87188|翻边|SSAC-292170-ECFAB


## 配置界面

In [5]:
RACEURL = 'http://116.85.5.40:9119'
API_INFERENCE = f'{RACEURL}/raceai/framework/inference'
API_POPMSG = f'{RACEURL}/raceai/private/popmsg'
MODEL_TASK = 'zmq.repnet_tf.inference'
MSGKEY = 'zmq.repnet_tf.eta'

device_list = [(name, mac) for mac, name, _ in devices]
date_list = oss_get_bypath(f'live/{device_list[0][1]}/')[-7:]
video_list = oss_get_video_list(date_list[-1][1])
task_list = oss_get_bypath(f'datasets/ladder/')
sample_list = oss_get_video_samples(task_list[0][1])

schema = {
    'type': 'page',
    'objs': [
        {
            'type': 'tab',
            'objs': [
                {
                    'name': 'CFG',
                    'objs': [
                        {
                            'type': 'H',
                            'objs': [
                                nbeasy_widget_string('task', 'Task', MODEL_TASK, width='30%', readonly=True),
                                nbeasy_widget_string('cfg.pigeon.msgkey', 'Mssage Key', 'zmq.repnet_tf.test', width='30%', readonly=True),
                                nbeasy_widget_string('cfg.pigeon.user_code', 'User Code', 'notebook', width='30%', readonly=True),
                            ]
                        },
                        {
                            'type': 'accordion',
                            'objs': [
                                {
                                    'name': 'Video Configuration',
                                    'objs': [
                                        {
                                            'type': 'navigation',
                                            'objs': [
                                                {
                                                    'type': 'H',
                                                    'name': 'Real Time',
                                                    'objs': [
                                                        nbeasy_widget_stringenum('_cfg.devices', 'Devices', 0, enums=device_list, width=400),
                                                        nbeasy_widget_stringenum('_cfg.date', 'Date', 0, enums=date_list, width=400),
                                                        nbeasy_widget_stringenum('_cfg.video', 'Video', 0, enums=video_list, width=400),
                                                    ]
                                                },
                                                {
                                                    'type': 'H',
                                                    'name': 'History Sample',
                                                    'objs': [
                                                        nbeasy_widget_stringenum('_cfg.types', 'Types', 0, enums=task_list, width=400),
                                                        nbeasy_widget_stringenum('_cfg.sample', 'Sample', 0, enums=sample_list, width=400),
                                                    ]
                                                }
                                            ]
                                            
                                        },
                                        {
                                            'type': 'V',
                                            'objs': [
                                                nbeasy_widget_bytes('cfg.video', '', 'http://', width=620),
                                                nbeasy_widget_video('_cfg.play_video', 'Play Video', '', format='url', width=720, height=480)
                                            ],
                                            'align_items': 'center'
                                        },
                                    ]
                                },
                                {
                                    'name': 'Score Configuration',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_bool('cfg.constant_speed', 'Constant Speed', False),
                                                nbeasy_widget_bool('cfg.median_filter', 'Median Filter', True),
                                                nbeasy_widget_bool('cfg.fully_periodic', 'Fully Periodic', False),
                                            ]
                                        }
                                    ]
                                },
                                {
                                    'name': 'Algo Configuration',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_int('cfg.batch_size', 'Batch Size', 20),
                                                nbeasy_widget_float('cfg.temperature', 'Temperature', 13.544),
                                                nbeasy_widget_intarray('cfg.strides', 'Strides', [3, 5, 7, 9, 11, 13], width=400),
                                            ]
                                        }
                                    ]
                                },
                                {
                                    'name': 'Optimize Configuration',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_booltrigger('cfg.rm_still', 'Remove Still', True, triggers=[
                                                    {},
                                                    {
                                                        'type': '_ignore_',
                                                        'objs': [
                                                            nbeasy_widget_float('cfg.area_rate_threshold', 'Area Threshold', 0.0005),
                                                        ]
                                                    }]),
                                                nbeasy_widget_booltrigger('_cfg.angle', 'Rotate Angle', False, triggers=[
                                                    {},
                                                    {
                                                        'type': '_ignore_',
                                                        'objs': [
                                                            nbeasy_widget_float('cfg.angle', 'Value', 45, min_=-180, max_=180)
                                                        ]
                                                    }
                                                ]),
                                                nbeasy_widget_booltrigger('_cfg.focus_box', 'Focus Box', False, triggers=[
                                                    {
                                                        'type': 'V',
                                                        'objs': [
                                                            nbeasy_widget_booltrigger('_cfg.center_rate', 'Center Rate', False, triggers=[
                                                                {
                                                                },
                                                                {
                                                                    'type': 'H',
                                                                    'objs': [
                                                                        nbeasy_widget_floatarray('cfg.center_rate', 'Value', [0.5, 0.9]),
                                                                    ]
                                                                }
                                                            ])
                                                        ]
                                                    },
                                                    {
                                                        'type': 'V',
                                                        'objs': [
                                                            nbeasy_widget_floatarray('cfg.focus_box', 'Rect', [0.15,0.05,0.85,0.95], width=300),
                                                            nbeasy_widget_int('cfg.focus_box_repnum', 'Replicate', 1, min_=1, max_=5),
                                                        ]
                                                    }]),
                                                nbeasy_widget_booltrigger('_cfg.black_box', 'Black Box', False, triggers=[
                                                    {},
                                                    {
                                                        'type': 'V',
                                                        'objs': [
                                                            nbeasy_widget_floatarray('cfg.black_box', 'Rect', [0,0,0,0], width=300),
                                                            nbeasy_widget_bool('cfg.black_overlay', 'Overlay', True)
                                                        ]
                                                    }]),
                                            ]
                                        },
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_booltrigger('cfg.best_stride_video', 'Save Stride Video', True, triggers=[{}, {
                                                    'type': '_ignore_',
                                                    'objs': [
                                                        nbeasy_widget_bool('cfg.osd_sims', 'OSD Sims', True),
                                                    ]}]),
                                                nbeasy_widget_bool('cfg.save_video', 'Save Full Video', False),
                                            ]
                                        },
                                    ]
                                }
                            ]
                        }, # end accordion
                    ]
                },
                {
                    'name': 'RUN',
                    'objs': [
                        {
                            'type': 'H',
                            'objs': [
                                nbeasy_widget_button('_cfg.btn_start', 'Start', width=100, style='primary'),
                                nbeasy_widget_button('_cfg.btn_stop', 'Stop', width=100, style='primary'),
                                nbeasy_widget_progressbar('_cfg.progressbar', 'Progress:', style='success', width=600)
                            ]
                        },
                        {
                            'type': 'H',
                            'objs': [
                                nbeasy_widget_text('_cfg.result_output', '', width=500, height=500),
                                {
                                    'type': 'V',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_stringenum('_cfg.video_mp4', 'MP4', 0, enums=[], description_width=40, width=200),
                                                nbeasy_widget_bytes('_cfg.video_url', '', 'http://', width=500),
                                            ]
                                        },
                                        nbeasy_widget_video('_cfg.play_stride_video', '', '', format='url', width=720, height=480),
                                    ]
                                },
                            ]
                        }
                    ]
                }
            ]
        }, # end tab
        {
            'type': 'jsdlink',
            'objs': [
                {
                    'source': ['_cfg.video', 'value'],
                    'target': ['cfg.video', 'value']
                },
                {
                    'source': ['_cfg.sample', 'value'],
                    'target': ['cfg.video', 'value']
                },
                {
                    'source': ['cfg.video', 'bvalue'],
                    'target': ['_cfg.play_video', 'value'],
                },
                {
                    'source': ['_cfg.video_mp4', 'value'],
                    'target': ['_cfg.video_url', 'value']
                },
                {
                    'source': ['_cfg.video_url', 'bvalue'],
                    'target': ['_cfg.play_stride_video', 'value'],
                }
            ] 
        }, # end jslink
        {
            'type': 'interactive',
            'objs': [
                {
                    'handler': 'xxx',
                    'params': {
                        'arg1': 'cfg.focus_box',
                        'arg2': 'cfg.black_box',
                    }
                }
            ]
        }, # end interactive
        {
            'type': 'observe',
            'objs': [
                {
                    'handler': 'get_date_list',
                    'params': {
                        'source': '_cfg.devices',
                        'target': '_cfg.date',
                    }
                },
                {
                    'handler': 'get_video_list',
                    'params': {
                        'source': '_cfg.date',
                        'target': '_cfg.video',
                    }
                },
                {
                    'handler': 'get_sample_list',
                    'params': {
                        'source': '_cfg.types',
                        'target': '_cfg.sample',
                    }
                },
            ]
        }, # end observe
        {
            'type': 'onclick',
            'objs': [
                {
                    'handler': 'start_inference',
                    'params': {
                        'source': '_cfg.btn_start',
                        'targets': ['_cfg.progressbar', '_cfg.result_output', '_cfg.video_mp4']
                    }
                },
                {
                    'handler': 'stop_inference',
                    'params': {
                        'source': '_cfg.btn_stop',
                        'targets': ['_cfg.btn_start']
                    }
                }
            ]
        }
    ]
}

with open('schema.json', 'w') as fw:
    fw.write(json.dumps(schema, indent=4, ensure_ascii=False))

## 解析配置

In [6]:
def start_inference(btn, w_bar, w_out, w_mp4):
    reqdata = btn.context.get_all_json()
    requests.get(url=f'{API_POPMSG}?key={MSGKEY}')
    result = json.loads(requests.post(url=API_INFERENCE, json=reqdata).text)
    w_out.value = json.dumps(reqdata, indent=4)
    if 'errno' in result:
        if result['errno'] < 0:
            w_out.value = json.dumps(result, indent=4)
            return
        
    btn.disabled = True
    def _run_result(btn, w_bar, w_out, w_mp4):
        cur_try = 0
        err_max = 60
        while cur_try < err_max:
            result = json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
            if len(result) == 0:
                time.sleep(1)
                cur_try += 1
                continue
            result = result[-1]
            cur_try = 0
            if result['errno'] != 0:
                w_out.value = json.dumps(result, indent=4)
                w_bar.value = 100.0
                break
            w_bar.value = int(result['progress'])
            if w_bar.value == 100.0:
                w_out.value = json.dumps(result, indent=4)
                options = []
                if 'stride_mp4' in result:
                    options.append(('stride_mp4', result['stride_mp4']))
                if 'target_mp4' in result:
                    options.append(('target_mp4', result['target_mp4']))
                w_mp4.options = options
        btn.disabled = False
    threading.Thread(target=_run_result, kwargs={
        'btn': btn,
        'w_bar': w_bar,
        'w_out': w_out,
        'w_mp4': w_mp4
    }).start()
    
def stop_inference(btn, start_button):
    start_button.disabled = False 
    
def get_date_list(source, oldval, newval, target):
    target.options = oss_get_bypath(f'live/{newval}/')[-7:]
    target.value = target.options[-1][1]
    
def get_video_list(source, oldval, newval, target):
    target.options = oss_get_video_list(newval)
    target.value = target.options[int(len(target.options) / 2)][1]
    
def get_sample_list(source, oldval, newval, target):
    target.options = oss_get_video_samples(newval)
    target.value = target.options[int(len(target.options) / 2)][1]

events = {
    'start_inference': start_inference,
    'stop_inference': stop_inference,
    'get_date_list': get_date_list,
    'get_video_list': get_video_list,
    'get_sample_list': get_sample_list,
}

In [7]:
parse_xls_report()
with open('schema.json', 'r') as fr:
    strs = fr.read().replace('zmq.repnet_tf.test', MSGKEY)
    schema = json.loads(strs)
# oss_remove_outputs()
easy = nbeasy_schema_parse(schema, events=events, debug=True);

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(HBox(children=(Text(value='zmq.repnet_…